# ENIGH Database

@roman

26 June, 2024

Code to create an ETL of the Enigh database, which is a database of the National Institute of Statistics and Geography (INEGI) of Mexico. The database contains information on the income and expenses of Mexican households.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm

In [2]:
# Settings
# show 100 columns in pandas
pd.set_option('display.max_columns', 500)

---
# Concentrado Hogar

The data diccionary is found [here](https://www.inegi.org.mx/rnm/index.php/catalog/685/data-dictionary/F28?file_name=concentradohogar)

## S1: Extract & Transform

In [3]:
# function to extract, transform and load each enigh file
def weighted_mean(df, cols, weight_col):
    return (df[cols].mul(df[weight_col], axis=0).sum() / df[weight_col].sum())


def weighted_mode(df, columns_to_mode, weighting_column):
    # Initialize a DataFrame to store the mode values
    mode_values = pd.DataFrame(columns=['Column', 'Weighted_Mode'])

    # Calculate weighted mode for each column in columns_to_mode
    for col in columns_to_mode:
        # Group by the column values and calculate weighted counts
        weighted_counts = df.groupby(col)[weighting_column].sum()

        # Find the index of maximum weighted count
        mode_index = weighted_counts.idxmax()

        # Get the mode value
        mode_value = df.loc[df[col] == mode_index, col].iloc[0]

        # Append mode value to mode_values DataFrame
        mode_values.loc[len(mode_values)] = [col, mode_value]

    # return a Series
    return mode_values.set_index('Column')['Weighted_Mode']


def get_statistics(df, cols_mode, cols_mean, weight_col):
    # s1: get weighted mode
    mode_values = weighted_mode(df, cols_mode, weight_col)

    # s2: get weighted mean
    mean_values = weighted_mean(df, cols_mean, weight_col)

    # s3: get total viviendas
    total_hogares = df[weight_col].sum()

    # return all the values in a pd series
    melt_values = pd.concat([mode_values, mean_values])
    melt_values['total_hogares'] = total_hogares

    return melt_values


def get_enigh_concentrado_hogar(file, cols_mode, cols_mean):
    # s1:read file
    df = pd.read_csv(file, na_values=[' '])
    
    # s2: subset and wrangle
    df = (
        df
        .query("foliohog == 1")
        .reset_index(drop=True)
        .assign(
            ubica_geo=lambda x: x["ubica_geo"].astype(str).str.zfill(5)
        )
    )

    # s3: get weighted mean
    table = (
        df
        .groupby("ubica_geo", as_index=False)
        .apply(
            lambda x: get_statistics(x, cols_mode, cols_mean, "factor"),
            include_groups=False
        )
    )
    
    return table


In [4]:
# get params
years_enigh = [2018, 2020, 2022]

cols_mode = [
    'est_dis', 'clase_hog', 'sexo_jefe',
]
cols_mean = [
    'tam_loc', 'est_socio',  'edad_jefe',
    'educa_jefe', 'tot_integ', 'hombres', 'mujeres',
    'mayores', 'menores', 'p12_64', 'p65mas', 'ocupados',
    'percep_ing', 'perc_ocupa', 'ing_cor', 'ingtrab',
    'trabajo', 'sueldos', 'horas_extr', 'comisiones',
    'aguinaldo', 'indemtrab', 'otra_rem', 'remu_espec',
    'negocio', 'noagrop', 'industria', 'comercio', 'servicios',
    'agrope', 'agricolas', 'pecuarios', 'reproducc', 'pesca',
    'otros_trab', 'rentas', 'utilidad', 'arrenda', 'transfer',
    'jubilacion', 'becas', 'donativos', 'remesas', 'bene_gob',
    'transf_hog', 'trans_inst', 'estim_alqu', 'otros_ing', 'gasto_mon',
    'alimentos', 'ali_dentro', 'cereales', 'carnes', 'pescado',
    'leche', 'huevo', 'aceites', 'tuberculo', 'verduras', 'frutas',
    'azucar', 'cafe', 'especias', 'otros_alim', 'bebidas', 'ali_fuera',
    'tabaco', 'vesti_calz', 'vestido', 'calzado', 'vivienda',
    'alquiler', 'pred_cons', 'agua', 'energia', 'limpieza', 'cuidados',
    'utensilios', 'enseres', 'salud', 'atenc_ambu', 'hospital',
    'medicinas', 'transporte', 'publico', 'foraneo', 'adqui_vehi',
    'mantenim', 'refaccion', 'combus', 'comunica', 'educa_espa',
    'educacion', 'esparci', 'paq_turist', 'personales', 'cuida_pers',
    'acces_pers', 'otros_gas', 'transf_gas', 'percep_tot', 'retiro_inv',
    'prestamos', 'otras_perc', 'ero_nm_viv', 'ero_nm_hog', 'erogac_tot',
    'cuota_viv', 'mater_serv', 'material', 'servicio', 'deposito',
    'prest_terc', 'pago_tarje', 'deudas', 'balance', 'otras_erog',
]

FILE_TEMPLATE = 'enigh{enigh_year}_ns_concentradohogar_csv.zip'
ENIGH_ROOT = "../../data/catalogues/adamuz_data/enigh_{enigh_year}/"

In [5]:
# get all tables
list_tables = []
for year in tqdm(years_enigh):
    # get file
    dir_file = ENIGH_ROOT.format(enigh_year=year)
    file = dir_file + FILE_TEMPLATE.format(enigh_year=year)

    # unzip
    os.system(f"unzip -o {file} -d {dir_file}")

    # get table
    table = get_enigh_concentrado_hogar(
        dir_file + 'concentradohogar.csv',
        cols_mode,
        cols_mean
        )

    # add year
    table["year"] = year

    # append
    list_tables.append(table)


  0%|          | 0/3 [00:00<?, ?it/s]

Archive:  ../../data/catalogues/adamuz_data/enigh_2018/enigh2018_ns_concentradohogar_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2018/concentradohogar.csv  
  inflating: ../../data/catalogues/adamuz_data/enigh_2018/nota_bases_datos_enigh2018_ns.txt  


 33%|███▎      | 1/3 [00:04<00:09,  4.64s/it]

Archive:  ../../data/catalogues/adamuz_data/enigh_2020/enigh2020_ns_concentradohogar_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2020/concentradohogar.csv  


 67%|██████▋   | 2/3 [00:09<00:04,  4.76s/it]

Archive:  ../../data/catalogues/adamuz_data/enigh_2022/enigh2022_ns_concentradohogar_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2022/concentradohogar.csv  


100%|██████████| 3/3 [00:14<00:00,  4.82s/it]


In [6]:
# concat
df_concentradohogar_all = (
    pd.concat(list_tables)
    .sort_values(["ubica_geo", "year"], ignore_index=True)
    )
df_concentradohogar_all.shape

(3218, 123)

## S2: EDA

In [7]:
# see sample
df_concentradohogar_all.head()

,ubica_geo,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares,year
0,01001,2.0,2.0,1.0,1.313242,2.830593,50.089514,6.479373,3.721097,1.780415,1.940683,2.999458,0.721640,2.691494,0.307964,1.803984,2.391824,1.774901,63956.301534,43070.626604,37697.351960,31202.010734,495.394570,1317.133005,2206.442086,252.169473,852.605979,1371.596112,3683.768570,3046.691033,415.123858,1268.234312,1363.332863,637.077537,0.783675,633.157600,3.136262,0.0,1689.506073,4540.471227,3606.090472,934.380755,9947.056421,5419.655226,159.891602,989.171367,295.724280,299.683788,1599.486996,1183.443163,6351.823899,46.323383,39119.615453,11954.944213,8940.815992,1409.190847,1791.346954,141.805814,1001.070483,256.830314,64.707445,116.593703,888.009195,411.459735,58.341930,74.699891,70.812195,1791.369493,864.577994,2928.156080,85.972141,1740.256643,1088.568451,651.688192,3114.482487,1146.301370,175.330648,644.507169,1148.343300,2429.241635,1727.998294,274.058488,427.184852,1639.883555,1227.447356,197.869469,214.566731,8381.883266,1171.234546,335.669626,1267.892256,3749.066387,355.853740,3393.212646,1858.020452,5514.063280,3462.857938,1320.678289,730.527053,2968.421188,2140.535891,117.545545,710.339753,1376.439186,4509.766233,906.226031,512.698217,1570.168785,10.959857,1509.713343,8311.227510,1231.299092,479.004866,209.606931,269.397936,2451.678069,257.427359,2074.223392,787.065174,316.650080,713.879477,234164.0,2018
1,01001,2.0,2.0,1.0,1.191967,2.848673,50.307018,6.518359,3.511532,1.670060,1.841472,2.877479,0.634053,2.578268,0.299212,1.696833,2.295080,1.642091,60912.502414,39994.510208,36177.707945,30166.188042,278.128582,943.278159,2202.367386,323.549054,1224.366678,1039.830043,2677.040577,2616.953301,378.597045,837.298788,1401.057468,60.087276,2.420698,57.607004,0.059573,0.0,1139.761686,3339.710125,2725.802014,613.908110,10562.410583,6146.652668,104.825597,1051.749094,361.698340,898.020269,1472.286234,527.178381,6937.108693,78.762806,37361.636895,12577.730151,10739.532593,1660.287784,1981.896889,180.714402,1105.435781,366.783710,92.778479,130.161860,1079.879732,523.272579,81.819202,84.717259,85.751375,2324.198133,1041.835407,1720.387090,117.810468,1351.179617,878.251851,472.927765,3523.484832,1395.231161,184.045724,789.587590,1154.620356,2287.033608,1552.910254,271.907183,462.216171,1778.678539,1277.235951,174.258678,327.183910,8257.743571,929.733346,119.353027,1820.589293,3162.720259,342.765384,2819.954876,2225.347646,3700.139892,2536.653595,776.041132,387.445165,2749.710175,1969.424468,112.600106,667.685601,1135.936510,4979.077693,1640.427443,570.140620,1558.225213,42.339486,1167.944931,10520.616231,1834.717706,532.231593,265.539319,266.692273,4694.411677,129.864933,1556.153613,757.984491,463.801093,551.451125,267473.0,2020
2,01001,2.0,2.0,1.0,1.260126,2.843422,50.853007,6.667584,3.485117,1.630896,1.854221,2.844748,0.640369,2.536844,0.307904,1.688633,2.319125,1.663681,85273.975500,54618.627742,46768.736235,3881

In [8]:
# shape
df_concentradohogar_all.shape

(3218, 123)

In [9]:
# see empty values
df_concentradohogar_all.isnull().sum()[df_concentradohogar_all.isnull().sum() > 0]

Series([], dtype: int64)

In [10]:
# count # of ubica_geo
df_concentradohogar_all["ubica_geo"].value_counts().value_counts()

count
3    664
1    502
2    362
Name: count, dtype: int64

In [11]:
# describe
df_concentradohogar_all.describe()

,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares,year
count,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000
mean,306.894655,2.024549,1.018334,3.126627,1.738642,51.777733,5.098020,3.626452,1.756526,1.869925,2.874127,0.752324,2.516746,0.357382,1.737205,2.361901,1.659605,42832.739771,27342.263322,21022.137858,18360.965488,142.262048,387.370905,918.352346,100.772285,375.463892,736.950893,5144.973552,3571.022166,965.558829,1307.249235,1298.214102,1573.951386,839.207739,648.441564,35.628280,50.673804,1175.151912,2560.332132,2155.669740,404.662392,8356.073000,2834.563251,69.764822,1111.201224,842.251982,1682.114427,1294.960880,521.216414,4540.223875,33.847443,26978.456638,10722.503389,9178.216533,1815.776905,1985.935418,191.479814,810.754164,399.759513,179.801073,158.835174,1166.499288,364.515518,132.162500,100.298404,85.964536,1004.160293,782.273934,1500.582380,43.704476,1089.014242,663.350024,425.664218,2143.950353,492.101690,105.836001,233.313465,1312.699197,1745.702951,1326.950768,154.707186,264.044997,1090.296698,805.042192,113.984523,171.269983,5031.318885,1135.877534,138.677962,654.763285,1912.605288,190.388716,1722.216572,1189.394816,2289.396380,1641.266439,485.301510,162.828431,2108.788971,1643.044461,70.585569,395.158941,757.484768,3404.853373,887.634839,423.186890,563.459168,41.937477,1488.634998,5091.985670,281.405813,382.341654,246.206399,136.135255,2710.776539,81.868545,453.112205,378.826458,479.237772,324.416685,32945.253263,2020.084525
std,145.922871,0.170082,0.134178,0.942440,0.

## S3: Explicit Nulls

In [12]:
# generate explicit nulls for ubica_geo and year
index_cols = ["ubica_geo", "year"]
new_index = pd.MultiIndex.from_product(
    [df_concentradohogar_all["ubica_geo"].unique(), df_concentradohogar_all["year"].unique()],
    names=index_cols
)

# reindex
df_concentradohogar_all = (
    df_concentradohogar_all
    .set_index(index_cols)
    .reindex(new_index)
    .reset_index()
    )

In [13]:
# count # of ubica_geo
df_concentradohogar_all["ubica_geo"].value_counts().value_counts()

count
3    1528
Name: count, dtype: int64

In [14]:
# see ubica geo 01004
df_concentradohogar_all.query("ubica_geo == '01004'")

,ubica_geo,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares
9,01004,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,01004,2020,11.0,2.0,1.0,4.0,2.0,53.095238,4.952381,4.666667,2.095238,2.571429,3.523810,1.142857,2.952381,0.571429,1.666667,3.095238,1.619048,50108.640000,29205.945714,20554.909048,16689.658571,16.304286,0.0,1425.112381,0.0,863.847619,1559.986190,6453.678571,3989.440000,0.0,58.229524,3931.210476,2464.238571,52.406667,2411.831905,0.0,0.0,2197.358095,0.0,0.0,0.0,17151.685238,6183.724286,58.229524,579.967619,3232.916667,2820.648571,2437.044762,1839.153810,3442.391429,308.617619,29510.594286,13492.470952,11801.460952,2150.163333,1509.784286,0.000000,1468.144762,726.722857,230.506667,200.199524,1711.491905,512.132381,245.506667,91.223333,83.874762,1893.665238,978.045238,1678.765238,12.244762,815.899524,440.392381,375.507143,2110.618571,69.124286,229.52381,320.000000,1491.970476,2548.000000,2275.535238,253.831429,18.633333,405.970000,376.810000,10.993333,18.166667,5659.382381,745.705238,10.946667,1271.738571,2076.111429,71.505238,2004.606190,1554.880476,1202.63,493.482857,476.228095,232.919048,2283.779524,1708.030476,31.210000,544.539048,991.843333,2058.328571,638.197619,698.756667,15.139048,0.0,706.235238,6272.042857,0.0,1411.489524,1276.396667,135.092857,4227.480000,451.396667,0.0,0.0,0.0,181.676667,1155.0
11,01004,2022,11.0,2.0,1.0,4.0,2.0,55.555556,4.166667,4.500000,2.111111,2.388889,3.333333,1.166667,2.833333,0.500000,1.777778,2.666667,1.722222,42181.936667,32637.283333,26569.804444,23682.403333,288.609444,0.0,1494.448333,0.0,179.971111,924.372222,4776.420556,1205.441667,0.0,41.331111,1164.110556,3570.978889,972.763889,2598.215000,0.0,0.0,1291.058333,0.0,0.0,0.0,6420.463333,0.000000,132.596667,1272.037222,379.229444,2604.846111,1673.525000,358.228889,3124.190000,0.000000,26342.328889,10408.444444,9682.019444,2974.238333,949.277778,117.856667,541.420556,427.138333,94.998889,210.711667,1291.761667,419.638889,47.142222,7.142778,14.285000,1331.424444,1254.982222,610.711667,115.713333,847.595556,425.861111,421.734444,2041.126667,166.666667,29.37500,423.333333,1421.751667,1961.920556,1568.127222,173.911667,219.881667,1284.121667,1086.679444,0.000000,197.442222,4915.933889,1071.417778,70.765000,0.000000,2943.806667,208.862778,2734.943889,829.944444,1176.49,657.202778,329.070000,190.217222,3333.295000,2838.871111,2.825556,491.598333,373.401111,637.091111,0.000000,0.000000,0.000000,0.0,637.091111,1127.683333,0.0,2.732222,2.732222,0.00

In [15]:
# fill na with strategy ffill and then bfill by ubica_geo
df_concentradohogar_all = (
    df_concentradohogar_all
    .sort_values(["ubica_geo", "year"])
    .groupby("ubica_geo")
    .apply(lambda x: x.ffill().bfill())
    .reset_index(drop=True)
    )

/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_36287/796339476.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.ffill().bfill())


In [16]:
# see ubica geo 01004
df_concentradohogar_all.query("ubica_geo == '01004'")

,ubica_geo,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares
9,01004,2018,11.0,2.0,1.0,4.0,2.0,53.095238,4.952381,4.666667,2.095238,2.571429,3.523810,1.142857,2.952381,0.571429,1.666667,3.095238,1.619048,50108.640000,29205.945714,20554.909048,16689.658571,16.304286,0.0,1425.112381,0.0,863.847619,1559.986190,6453.678571,3989.440000,0.0,58.229524,3931.210476,2464.238571,52.406667,2411.831905,0.0,0.0,2197.358095,0.0,0.0,0.0,17151.685238,6183.724286,58.229524,579.967619,3232.916667,2820.648571,2437.044762,1839.153810,3442.391429,308.617619,29510.594286,13492.470952,11801.460952,2150.163333,1509.784286,0.000000,1468.144762,726.722857,230.506667,200.199524,1711.491905,512.132381,245.506667,91.223333,83.874762,1893.665238,978.045238,1678.765238,12.244762,815.899524,440.392381,375.507143,2110.618571,69.124286,229.52381,320.000000,1491.970476,2548.000000,2275.535238,253.831429,18.633333,405.970000,376.810000,10.993333,18.166667,5659.382381,745.705238,10.946667,1271.738571,2076.111429,71.505238,2004.606190,1554.880476,1202.63,493.482857,476.228095,232.919048,2283.779524,1708.030476,31.210000,544.539048,991.843333,2058.328571,638.197619,698.756667,15.139048,0.0,706.235238,6272.042857,0.0,1411.489524,1276.396667,135.092857,4227.480000,451.396667,0.0,0.0,0.0,181.676667,1155.0
10,01004,2020,11.0,2.0,1.0,4.0,2.0,53.095238,4.952381,4.666667,2.095238,2.571429,3.523810,1.142857,2.952381,0.571429,1.666667,3.095238,1.619048,50108.640000,29205.945714,20554.909048,16689.658571,16.304286,0.0,1425.112381,0.0,863.847619,1559.986190,6453.678571,3989.440000,0.0,58.229524,3931.210476,2464.238571,52.406667,2411.831905,0.0,0.0,2197.358095,0.0,0.0,0.0,17151.685238,6183.724286,58.229524,579.967619,3232.916667,2820.648571,2437.044762,1839.153810,3442.391429,308.617619,29510.594286,13492.470952,11801.460952,2150.163333,1509.784286,0.000000,1468.144762,726.722857,230.506667,200.199524,1711.491905,512.132381,245.506667,91.223333,83.874762,1893.665238,978.045238,1678.765238,12.244762,815.899524,440.392381,375.507143,2110.618571,69.124286,229.52381,320.000000,1491.970476,2548.000000,2275.535238,253.831429,18.633333,405.970000,376.810000,10.993333,18.166667,5659.382381,745.705238,10.946667,1271.738571,2076.111429,71.505238,2004.606190,1554.880476,1202.63,493.482857,476.228095,232.919048,2283.779524,1708.030476,31.210000,544.539048,991.843333,2058.328571,638.197619,698.756667,15.139048,0.0,706.235238,6272.042857,0.0,1411.489524,1276.396667,135.092857,4227.480000,451.396667,0.0,0.0,0.0,181.676667,1155.0
11,01004,2022,11.0,2.0,1.0,4.0,2.0,55.555556,4.166667,4.500000,2.111111,2.388889,3.333333,1.166667,2.833333,0.500000,1.777778,2.666667,1.722222,42181.936667,32637.283333,26569.804444,23682.403333,288.609444,0.0,1494.448333,0.0,179.971111,924.372222,4776.420556,1205.441667,0.0,41.331111,1164.110556,3570.978889,972.763889,2598.215000,0.0,0.0,1291.058333,0.0,0.0,0.0,6420.463333,0.000000,132.596667,1272.037222,379.2294

## S4: Save

In [17]:
# see shape
df_concentradohogar_all.shape

(4584, 123)

In [18]:
# save
dir_save = "../../data/interim/"
df_concentradohogar_all.to_csv(dir_save + "concentrado_hogar_enigh.csv")

---
# Vivienda

INEGI's data diccionary is found [here](https://www.inegi.org.mx/rnm/index.php/catalog/685/data-dictionary/F13?file_name=viviendas)

Notes:
- A Hogar can have multiple viviendas

## S1: Extract & Transform

In [191]:
# function to extract, transform and load each enigh file
def weighted_mode(df, columns_to_mode, weighting_column):
    # Initialize a DataFrame to store the mode values
    mode_values = pd.DataFrame(columns=['Column', 'Weighted_Mode'])

    # Calculate weighted mode for each column in columns_to_mode
    for col in columns_to_mode:
        # Group by the column values and calculate weighted counts
        weighted_counts = df.groupby(col)[weighting_column].sum()

        # Find the index of maximum weighted count
        mode_index = weighted_counts.idxmax()

        # Get the mode value
        mode_value = df.loc[df[col] == mode_index, col].iloc[0]

        # Append mode value to mode_values DataFrame
        mode_values.loc[len(mode_values)] = [col, mode_value]

    # return a Series
    return mode_values.set_index('Column')['Weighted_Mode']


def weighted_mean(df, cols, weight_col):
    return (df[cols].mul(df[weight_col], axis=0).sum() / df[weight_col].sum())


def get_statistics(df, cols_mode, cols_mean, weight_col):
    # s1: get weighted mode
    mode_values = weighted_mode(df, cols_mode, weight_col)

    # s2: get weighted mean
    mean_values = weighted_mean(df, cols_mean, weight_col)

    # s3: get total viviendas
    total_viviendas = df[weight_col].sum()

    # return all the values in a pd series
    melt_values = pd.concat([mode_values, mean_values])
    melt_values['total_viviendas'] = total_viviendas

    return melt_values


def get_enigh_vivienda(file, cols_mode, cols_mean):
    # s1:read file
    df = pd.read_csv(file, na_values=[' '])

    # s2: wrangle
    yes_no_columns = [
        'cocina',
        'cocina_dor',
        'excusado',
        'uso_compar',
        'biodigest',
        'combustible',
        'estufa_chi',
        'lavadero',
        'fregadero',
        'regadera',
        'tinaco_azo',
        'cisterna',
        'pileta',
        'calent_sol',
        'calent_gas',
        'medidor_luz',
        'bomba_agua',
        'tanque_gas',
        'aire_acond',
        'calefacc',
        'pago_mesp',
        'viv_usada'
    ]
    # map 1 for 1 and 2 for 0 (yes and no)
    df[yes_no_columns] = df[yes_no_columns].replace({2: 0})
    # ubica_geo
    df["ubica_geo"] = df["ubica_geo"].astype(str).str.zfill(5)

    # s3: get statistics
    table = (
        df
        .groupby("ubica_geo", as_index=False)
        .apply(
            lambda x: get_statistics(x, cols_mode, cols_mean, "factor"),
            include_groups=False
        )
    )

    return table


In [192]:
# get params
years_enigh = [2018, 2020, 2022]
cols_mode = [
    'mat_pared', 'mat_techos', 'mat_pisos', 'disp_agua',
    'combustible', 'eli_basura', 'tenencia', 'tipo_adqui',
    'tipo_adqui', 'tipo_finan', 'escrituras', 'disp_elect',
    'tipo_viv'
]
cols_mean = [
    'regadera', 'pago_mesp', 'tinaco_azo',
    'lavadero', 'procaptar', 'tot_resid', 'antiguedad',
    'bano_comp', 'calent_sol', 'cocina',
    'aire_acond', 'tot_hom', 'cocina_dor', 'renta',
    'fregadero', 'focos_inca', 'sanit_agua',
    'uso_compar', 'medidor_luz', 'est_socio',
    'tot_muj', 'dotac_agua', 'bano_regad', 'estufa_chi',
    'tam_loc', 'tanque_gas', 'focos_ahor',
    'cisterna', 'cuart_dorm', 'drenaje', 'excusado', 'pileta',
    'num_cuarto', 'calent_gas', 'calefacc',
    'bano_excus', 'pago_viv',
    'bomba_agua', 'viv_usada', 'biodigest', 'tot_hog',
    'estim_pago'
]

FILE_TEMPLATE = 'enigh{enigh_year}_ns_viviendas_csv.zip'
ENIGH_ROOT = "../../data/catalogues/adamuz_data/enigh_{enigh_year}/"

In [193]:
# get all tables
list_tables = []
for year in tqdm(years_enigh):
    # get file
    dir_file = ENIGH_ROOT.format(enigh_year=year)
    file = dir_file + FILE_TEMPLATE.format(enigh_year=year)

    # unzip
    os.system(f"unzip -o {file} -d {dir_file}")

    # get table
    table = get_enigh_vivienda(dir_file + 'viviendas.csv', cols_mode, cols_mean)

    # add year
    table["year"] = year

    # append
    list_tables.append(table)

  0%|          | 0/3 [00:00<?, ?it/s]

Archive:  ../../data/catalogues/adamuz_data/enigh_2018/enigh2018_ns_viviendas_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2018/viviendas.csv  
  inflating: ../../data/catalogues/adamuz_data/enigh_2018/nota_bases_datos_enigh2018_ns.txt  


/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_36287/892717475.py:47: DtypeWarning: Columns (4,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, na_values=[' '])
 33%|███▎      | 1/3 [00:14<00:28, 14.37s/it]

Archive:  ../../data/catalogues/adamuz_data/enigh_2020/enigh2020_ns_viviendas_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2020/viviendas.csv  


/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_36287/892717475.py:47: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, na_values=[' '])
 67%|██████▋   | 2/3 [00:29<00:14, 14.83s/it]

Archive:  ../../data/catalogues/adamuz_data/enigh_2022/enigh2022_ns_viviendas_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2022/viviendas.csv  


/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_36287/892717475.py:47: DtypeWarning: Columns (4,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, na_values=[' '])
100%|██████████| 3/3 [00:45<00:00, 15.22s/it]


In [194]:
# concat
df_viviendas_all = (
    pd.concat(list_tables)
    .sort_values(["ubica_geo", "year"], ignore_index=True)
    )
df_viviendas_all

,ubica_geo,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas,year
0,01001,8.0,10.0,3.0,1.0,3.0,3.0,4.0,1.0,1.0,1,1.0,1,1,0.945602,0.362238,0.782853,0.895975,0.0,3.781875,19.145343,1.265395,0.263397,0.979702,0.015126,1.806226,0.023304,385.384175,0.882548,1.506320,1.069981,0.029129,0.972374,2.830593,1.975650,1.091257,0.010228,0.001665,1.313242,0.146555,7.696495,0.493936,2.460400,1.044960,0.996631,0.133018,4.529099,0.515284,0.010715,0.266219,442.032669,0.416913,0.166781,0.014665,1.019717,2165.050435,234164,2018
1,01001,8.0,10.0,3,1.0,3.0,3.0,4.0,1.0,1.0,1,1.0,1,1,0.962908,0.385572,0.805143,0.903710,0.0,3.555936,18.578944,1.301716,0.287558,0.988414,0.017228,1.693341,0.020858,484.117574,0.898580,1.067147,1.066227,0.022514,0.982148,2.848673,1.862595,1.177386,0.007960,0.003885,1.191967,0.152516,7.850508,0.475506,2.355871,1.009949,0.996949,0.098937,4.408034,0.492554,0.012072,0.198125,670.729401,0.391329,0.161022,0.010222,1.016633,2361.336658,267473,2020
2,01001,8.0,10.0,3,1.0,3,3.0,4.0,1.0,1.0,1,1.0,1,1,0.970420,0.386612,0.784273,0.905320,0.0,3.514820,20.415628,1.364741,0.350969,0.986751,0.027798,1.645770,0.018158,594.349359,0.916743,0.690979,1.054902,0.015830,0.972788,2.843422,1.869050,1.180679,0.005229,0.004247,1.260126,0.167579,8.915832,0.438922,2.393433,1.033536,0.998091,0.120005,4.433587,0.439448,0.017268,0.239865,711.650391,0.403757,0.179857,0.029685,1.010339,3013.785845,267713,2022
3,01002,8.0,10.0,2.0,1.0,3.0,3.0,4.0,3.0,3.0,5,1.0,1,1,0.858142,0.033360,0.825390,0.900527,0.0,3.604238,19.959440,0.925979,0.559116,0.966640,0.000000,1.983979,0.110221,35.956195,0.604847,2.070980,1.075745,0.064997,0.978503,2.000000,1.620260,1.121476,0.032752,0.000000,3.502535,0.193673,3.612148,0.108497,2.040256,1.031637,0.989252,0.097141,3.856520,0.383695,0.000000,0.225918,0.000000,0.066112,0.044109,0.010748,1.000000,1048.428311,9862,2018
4,01002,8.0,10.0,2,1.0,3.0,3.0,4.0,3.0,3.0,5,1.0,1,1,0.835095,0.026823,0.828621,0.833510,0.0,4.468816,22.767310,0.884514,0.607426,0.990618,0.008060,2.112183,0.018763,20.989693,0.569371,2.511364,1.166226,0.051401,0.951506,2.000000,2.356633,2.066332,0.021934,0.021010,4.000000,0.085359,4.049154,0.215777,2.379889,1.227669,0.979651,0.229783,4.209567,0.219212,0.008060,0.231501,0.000000,0.203356,0.029070,0.000000,1.027484,1159.791226,7568,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,32056,8.0,10.0,3.0,1.0,3.0,1.0,4.0,1.0,1.0,5,1.0,1,1,0.988622,0.307692,0.976436,0.669133,0.0,3.634024,20.698178,1.414077,0.417638,0.993963,0.000000,1.681708,0.000000,435.273334,0.899652,1.926805,1.035972,0.006065,0.993434,3.121463,1.952316,2.024788,0.011963,0.000000,1.000000,0.240840,7.018640,0.221616,2.378717,1.000000,1.000000,0.054389,4.806148,0.810238,0.000000,0.247016,363.107525,0.197357,0.084880,0.005648,1.010433,2008.997079,35945,2018
3214,32056,8.0,10.0,3,1.0,3.0,1.0,4.0,2.0,2.0,5,1.0,1,1,0.914939,0.214437,0.927101,0.696612,0.0,3.529814,21.250086,1.260574,0.583608,0.980378,0.014230,1.642572,0.019228,358.733813,0.821385,1.129967,1.119824,0.019942,0.968511,2.869344,1.887242,1.902826,0.018415,0.005072,1.304299,0.267098,7.814270,0.217613,2.318750,1.071003,0.994928,0.135900,4.486508,0.682087,0.014058,0.343468,295.051455,0.215742,0.058841,0.004481,1.021641,2243.818012,40618,2020
3215,32056,8.0,10.0,3.0,1.0,3.0,1.0,4.0,3.0,3.0,5,1.0,1,1,0.

## S2: EDA

In [195]:
# see sample
df_viviendas_all.head()

,ubica_geo,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas,year
0,01001,8.0,10.0,3.0,1.0,3.0,3.0,4.0,1.0,1.0,1,1.0,1,1,0.945602,0.362238,0.782853,0.895975,0.0,3.781875,19.145343,1.265395,0.263397,0.979702,0.015126,1.806226,0.023304,385.384175,0.882548,1.506320,1.069981,0.029129,0.972374,2.830593,1.975650,1.091257,0.010228,0.001665,1.313242,0.146555,7.696495,0.493936,2.460400,1.044960,0.996631,0.133018,4.529099,0.515284,0.010715,0.266219,442.032669,0.416913,0.166781,0.014665,1.019717,2165.050435,234164,2018
1,01001,8.0,10.0,3,1.0,3.0,3.0,4.0,1.0,1.0,1,1.0,1,1,0.962908,0.385572,0.805143,0.903710,0.0,3.555936,18.578944,1.301716,0.287558,0.988414,0.017228,1.693341,0.020858,484.117574,0.898580,1.067147,1.066227,0.022514,0.982148,2.848673,1.862595,1.177386,0.007960,0.003885,1.191967,0.152516,7.850508,0.475506,2.355871,1.009949,0.996949,0.098937,4.408034,0.492554,0.012072,0.198125,670.729401,0.391329,0.161022,0.010222,1.016633,2361.336658,267473,2020
2,01001,8.0,10.0,3,1.0,3,3.0,4.0,1.0,1.0,1,1.0,1,1,0.970420,0.386612,0.784273,0.905320,0.0,3.514820,20.415628,1.364741,0.350969,0.986751,0.027798,1.645770,0.018158,594.349359,0.916743,0.690979,1.054902,0.015830,0.972788,2.843422,1.869050,1.180679,0.005229,0.004247,1.260126,0.167579,8.915832,0.438922,2.393433,1.033536,0.998091,0.120005,4.433587,0.439448,0.017268,0.239865,711.650391,0.403757,0.179857,0.029685,1.010339,3013.785845,267713,2022
3,01002,8.0,10.0,2.0,1.0,3.0,3.0,4.0,3.0,3.0,5,1.0,1,1,0.858142,0.033360,0.825390,0.900527,0.0,3.604238,19.959440,0.925979,0.559116,0.966640,0.000000,1.983979,0.110221,35.956195,0.604847,2.070980,1.075745,0.064997,0.978503,2.000000,1.620260,1.121476,0.032752,0.000000,3.502535,0.193673,3.612148,0.108497,2.040256,1.031637,0.989252,0.097141,3.856520,0.383695,0.000000,0.225918,0.000000,0.066112,0.044109,0.010748,1.000000,1048.428311,9862,2018
4,01002,8.0,10.0,2,1.0,3.0,3.0,4.0,3.0,3.0,5,1.0,1,1,0.835095,0.026823,0.828621,0.833510,0.0,4.468816,22.767310,0.884514,0.607426,0.990618,0.008060,2.112183,0.018763,20.989693,0.569371,2.511364,1.166226,0.051401,0.951506,2.000000,2.356633,2.066332,0.021934,0.021010,4.000000,0.085359,4.049154,0.215777,2.379889,1.227669,0.979651,0.229783,4.209567,0.219212,0.008060,0.231501,0.000000,0.203356,0.029070,0.000000,1.027484,1159.791226,7568,2020


In [196]:
# shape
df_viviendas_all.shape

(3218, 58)

In [197]:
# see empty values
df_viviendas_all.isnull().sum()[df_viviendas_all.isnull().sum() > 0]

Series([], dtype: int64)

In [198]:
# count # of ubica_geo
df_viviendas_all["ubica_geo"].value_counts().value_counts()

count
3    664
1    502
2    362
Name: count, dtype: int64

In [199]:
# describe
df_viviendas_all.describe()

,mat_pared,mat_techos,disp_agua,eli_basura,tenencia,tipo_adqui,tipo_adqui,escrituras,disp_elect,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas,year
count,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000
mean,7.872281,8.279366,1.603170,1.633002,3.943443,2.169049,2.169049,1.353014,1.006837,0.632663,0.116859,0.539838,0.833896,0.072518,3.680122,19.966795,0.717635,0.131332,0.907696,0.107184,1.782540,0.060955,168.136975,0.468555,1.124025,1.461249,0.073366,0.919543,1.738642,1.897582,1.527458,0.043776,0.085863,3.126627,0.078632,5.008371,0.152714,2.017363,1.704186,0.967612,0.446672,3.619212,0.270530,0.020671,0.479583,105.376335,0.223503,0.057935,0.023900,1.016293,1546.849737,32945.253263,2020.084525
std,0.428437,2.956521,1.409176,1.206857,0.373886,0.652590,0.652590,0.760988,0.153550,0.290137,0.137805,0.281555,0.204577,0.250589,0.578878,5.926552,0.388612,0.191552,0.110453,0.204979,0.319315,0.058519,321.772576,0.283027,0.738890,0.411315,0.071763,0.112973,0.599341,0.332447,0.764325,0.066133,0.157747,0.942440,0.103955,1.836917,0.174639,0.275016,0.771586,0.066174,0.261569,0.583695,0.249719,0.065202,0.275174,243.101552,0.207830,0.067028,0.047719,0.030076,899.843595,60317.776856,1.624437
min,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.823529,3.733333,0.000000,0.000000,0.187500,0.000000,0.823529,0.000000,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,1.000000,0.500000,0.000000,0.000000,0.000000,1.000000,0.000000,0.032258,0.000000,1.055556,1.000000,0.142857,0.000000,1.684211,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,346.000000,596.000000,2018.000000
25%,8.000000,8.000000,1.000000,1.000000,4.000000,2.000000,2.000000,1.000000,1.000000,0.421113,0.000000,0.300000,0.773755,0.000000,3.315789,15.987030,0.425251,0.000000,0.875000,0.000000,1.578024,0.020557,0.000000,0.227002,0.565231,1.150000,0.023877,0.898095,1.000000,1.686838,1.000501,0.000000,0.000000,2.516317,0.000000,3.782784,0.029041,1.833333,1.086957,0.956431,0.225509,3.244407,0.045455,0.000000,0.252286,0.000000,0.052632,0.000000,0.000000,1.000000,1039.152261,7035.750000,2018.000000
50%,8.000000,10.000000,1.000000,1.000000,4.000000,2.000000,2.000000,1.000000,1.000000,0.697116,0.063959,0.580633,0.918696,0.000000,3.619048,19.370338,0.765071,0.029813,0.947368,0.000000,1.750978,0.048585,60.000000,0.462269,0.992645,1.352941,0.052632,0.954164,2.000000,1.869565,1.262893,0.021841,0.023276,3.298769,0.045455,4.947368,0.092385,2.017716,1.423005,0.997983,0.437706,3.631579,0.210526,0.000000,0.445595,0.000000,0.166667,0.043478,0.000000,1.000000,1340.976532,13650.000000,2020.000000
75%,8.000000,10.000000,2.000000,1.000000,4.000000,3.000000,3.000000,1.000000,1.000000,0.878542,0.190476,0.786549,0.970587,0.000000,3.991653,23.232315,1.018641,0.203478,0.984959,0.095625,1.954545,0.086957,221.486058,0.700000,1.565217,1.666667,0.100950,0.992186,2.000000,2.076226,1.904762,0.055767,0.094195,4.000000,0.111713,6.126788,0.217391,2.195970,2.110828,1.0000

## S3: Explicit Nulls

In [200]:
# generate explicit nulls for ubica_geo and year
index_cols = ["ubica_geo", "year"]
new_index = pd.MultiIndex.from_product(
    [df_viviendas_all["ubica_geo"].unique(), df_viviendas_all["year"].unique()],
    names=index_cols
)

# reindex
df_viviendas_all = (
    df_viviendas_all
    .set_index(index_cols)
    .reindex(new_index)
    .reset_index()
    )

In [201]:
# count # of ubica_geo
df_viviendas_all["ubica_geo"].value_counts().value_counts()

count
3    1528
Name: count, dtype: int64

In [202]:
# see ubica geo 01004
df_viviendas_all.query("ubica_geo == '01004'")

,ubica_geo,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas
9,01004,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,01004,2020,8.0,10.0,2,1.0,3.0,3.0,4.0,3.0,3.0,5,1.0,1.0,1,0.904762,0.047619,0.809524,0.952381,0.0,4.666667,28.666667,1.000000,0.333333,0.904762,0.0,2.095238,0.000000,23.809524,0.666667,1.571429,1.190476,0.047619,0.952381,2.0,2.571429,1.142857,0.047619,0.0,4.0,0.047619,5.047619,0.333333,2.666667,1.000000,1.0,0.238095,4.428571,0.285714,0.000000,0.142857,0.0,0.619048,0.0,0.0,1.0,1185.714286,1155.0
11,01004,2022,8.0,10.0,2,1.0,3,3.0,4.0,3.0,3.0,5,1.0,1.0,1,0.833333,0.055556,0.666667,0.666667,0.0,4.500000,15.277778,0.833333,0.222222,0.944444,0.0,2.111111,0.055556,55.555556,0.611111,0.555556,1.277778,0.111111,0.888889,2.0,2.388889,1.000000,0.000000,0.0,4.0,0.000000,4.333333,0.111111,2.111111,1.277778,1.0,0.222222,3.611111,0.166667,0.055556,0.333333,0.0,0.111111,0.0,0.0,1.0,1072.222222,1494.0


In [203]:
# fill na with strategy ffill and then bfill by ubica_geo
df_viviendas_all = (
    df_viviendas_all
    .sort_values(["ubica_geo", "year"])
    .groupby("ubica_geo")
    .apply(lambda x: x.ffill().bfill())
    .reset_index(drop=True)
    )

/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_36287/2231233023.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .apply(lambda x: x.ffill().bfill())
/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_36287/2231233023.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.ffill().bfill())


In [204]:
# see ubica geo 01004
df_viviendas_all.query("ubica_geo == '01004'")

,ubica_geo,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas
9,01004,2018,8.0,10.0,2,1.0,3.0,3.0,4.0,3.0,3.0,5,1.0,1.0,1,0.904762,0.047619,0.809524,0.952381,0.0,4.666667,28.666667,1.000000,0.333333,0.904762,0.0,2.095238,0.000000,23.809524,0.666667,1.571429,1.190476,0.047619,0.952381,2.0,2.571429,1.142857,0.047619,0.0,4.0,0.047619,5.047619,0.333333,2.666667,1.000000,1.0,0.238095,4.428571,0.285714,0.000000,0.142857,0.0,0.619048,0.0,0.0,1.0,1185.714286,1155.0
10,01004,2020,8.0,10.0,2,1.0,3.0,3.0,4.0,3.0,3.0,5,1.0,1.0,1,0.904762,0.047619,0.809524,0.952381,0.0,4.666667,28.666667,1.000000,0.333333,0.904762,0.0,2.095238,0.000000,23.809524,0.666667,1.571429,1.190476,0.047619,0.952381,2.0,2.571429,1.142857,0.047619,0.0,4.0,0.047619,5.047619,0.333333,2.666667,1.000000,1.0,0.238095,4.428571,0.285714,0.000000,0.142857,0.0,0.619048,0.0,0.0,1.0,1185.714286,1155.0
11,01004,2022,8.0,10.0,2,1.0,3,3.0,4.0,3.0,3.0,5,1.0,1.0,1,0.833333,0.055556,0.666667,0.666667,0.0,4.500000,15.277778,0.833333,0.222222,0.944444,0.0,2.111111,0.055556,55.555556,0.611111,0.555556,1.277778,0.111111,0.888889,2.0,2.388889,1.000000,0.000000,0.0,4.0,0.000000,4.333333,0.111111,2.111111,1.277778,1.0,0.222222,3.611111,0.166667,0.055556,0.333333,0.0,0.111111,0.0,0.0,1.0,1072.222222,1494.0


## S4: Save

In [205]:
# see shape
df_viviendas_all.shape

(4584, 58)

In [206]:
# save
dir_save = "../../data/interim/"
df_viviendas_all.to_csv(dir_save + "viviendas_enigh.csv")

---
# Join with Properties

## S1: Load & Transform

In [207]:
# read parquet
gdf_properties = pd.read_parquet("../../data/interim/cleaned_data_s4.parquet")

# get original columns
original_columns = gdf_properties.columns

# see num of rows
gdf_properties.shape

(852931, 41)

In [208]:
# see columns cve ent & cve mun
gdf_properties[["id_entidad_f", "id_municipio"]]

,id_entidad_f,id_municipio
0,09,003
1,15,058
2,15,121
3,09,003
4,15,058
...,...,...
853028,31,041
853029,31,050
853030,31,050
853031,31,050


In [209]:
# create 'ubica_geo' column
gdf_properties["ubica_geo"] = (
    gdf_properties["id_entidad_f"].astype(str).str.zfill(2)
    + gdf_properties["id_municipio"].astype(str).str.zfill(3)
    )

# see some values
gdf_properties["ubica_geo"]

0         09003
1         15058
2         15121
3         09003
4         15058
          ...  
853028    31041
853029    31050
853030    31050
853031    31050
853032    31050
Name: ubica_geo, Length: 852931, dtype: object

In [210]:
# get year of fecha avaluo and floor every 2 years
gdf_properties["year_enigh"] = (
    gdf_properties["fecha_avaluo"].dt.year
    // 2 * 2
    )

# describe
gdf_properties["year_enigh"].describe()

count    852931.000000
mean       2020.196398
std           1.485187
min        2018.000000
25%        2020.000000
50%        2020.000000
75%        2022.000000
max        2022.000000
Name: year_enigh, dtype: float64

## S2: Join with Concentrado Hogar

In [211]:
# join with df_concentradohogar_all
gdf_properties = (
    gdf_properties
    .merge(
        df_concentradohogar_all,
        left_on=["ubica_geo", "year_enigh"],
        right_on=["ubica_geo", "year"],
        how="left"
        )
    )

# see columns
gdf_properties.shape

(852931, 166)

In [212]:
# see size
gdf_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852931 entries, 0 to 852930
Columns: 166 entries, property_id to cve_ent
dtypes: datetime64[ns](1), float64(129), int32(2), int64(24), object(10)
memory usage: 1.0+ GB


In [213]:
# see if there are columns with "_y"
gdf_properties.filter(like="_x").columns

Index([], dtype='object')

In [214]:
# see how many nan values 
gdf_properties.loc[:, df_concentradohogar_all.columns].isnull().sum()

ubica_geo        0
year             0
est_dis          0
clase_hog        0
sexo_jefe        0
                ..
deudas           0
balance          0
otras_erog       0
total_hogares    0
cve_ent          0
Length: 124, dtype: int64

In [215]:
# see how many nan values 
gdf_properties.loc[:, df_concentradohogar_all.columns].isnull().mean()

ubica_geo        0.0
year             0.0
est_dis          0.0
clase_hog        0.0
sexo_jefe        0.0
                ... 
deudas           0.0
balance          0.0
otras_erog       0.0
total_hogares    0.0
cve_ent          0.0
Length: 124, dtype: float64

## S3: Join with Vivienda

In [216]:
# join with df_viviendas_all
gdf_properties = (
    gdf_properties
    .drop(columns=["year", "tam_loc", "est_socio"])
    .merge(
        df_viviendas_all,
        left_on=["ubica_geo", "year_enigh"],
        right_on=["ubica_geo", "year"],
        how="left"
        )
    )

# see columns
gdf_properties.shape

(852931, 220)

In [217]:
# see size
gdf_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852931 entries, 0 to 852930
Columns: 220 entries, property_id to total_viviendas
dtypes: datetime64[ns](1), float64(180), int32(1), int64(24), object(14)
memory usage: 1.4+ GB


In [218]:
# see if there are columns with "_y"
gdf_properties.filter(like="_x").columns

Index([], dtype='object')

In [219]:
# see how many nan values 
gdf_properties.loc[:, df_viviendas_all.columns].isnull().sum()

ubica_geo             0
year               2592
mat_pared          2592
mat_techos         2592
mat_pisos          2592
disp_agua          2592
combustible        2592
eli_basura         2592
tenencia           2592
tipo_adqui         2592
tipo_adqui         2592
tipo_adqui         2592
tipo_adqui         2592
tipo_finan         2592
escrituras         2592
disp_elect         2592
tipo_viv           2592
regadera           2592
pago_mesp          2592
tinaco_azo         2592
lavadero           2592
procaptar          2592
tot_resid          2592
antiguedad         2592
bano_comp          2592
calent_sol         2592
cocina             2592
aire_acond         2592
tot_hom            2592
cocina_dor         2592
renta              2592
fregadero          2592
focos_inca         2592
sanit_agua         2592
uso_compar         2592
medidor_luz        2592
est_socio          2592
tot_muj            2592
dotac_agua         2592
bano_regad         2592
estufa_chi         2592
tam_loc         

In [220]:
# see how many nan values 
gdf_properties.loc[:, df_viviendas_all.columns].isnull().mean()

ubica_geo          0.000000
year               0.003039
mat_pared          0.003039
mat_techos         0.003039
mat_pisos          0.003039
disp_agua          0.003039
combustible        0.003039
eli_basura         0.003039
tenencia           0.003039
tipo_adqui         0.003039
tipo_adqui         0.003039
tipo_adqui         0.003039
tipo_adqui         0.003039
tipo_finan         0.003039
escrituras         0.003039
disp_elect         0.003039
tipo_viv           0.003039
regadera           0.003039
pago_mesp          0.003039
tinaco_azo         0.003039
lavadero           0.003039
procaptar          0.003039
tot_resid          0.003039
antiguedad         0.003039
bano_comp          0.003039
calent_sol         0.003039
cocina             0.003039
aire_acond         0.003039
tot_hom            0.003039
cocina_dor         0.003039
renta              0.003039
fregadero          0.003039
focos_inca         0.003039
sanit_agua         0.003039
uso_compar         0.003039
medidor_luz        0

## S4: See which ubica_geo have nan values

In [60]:
# see how many nan values in ubica_geo
nan_ubica_geo = (
    gdf_properties
    .loc[:, df_viviendas_all.columns]
    .groupby("ubica_geo")
    .apply(lambda x: x['tipo_viv'].isnull().sum())
    .to_frame()
    .rename(columns={0: "nan_values"})
    .query("nan_values > 0")
    .reset_index()
)
nan_ubica_geo

/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_36287/77283165.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x['tipo_viv'].isnull().sum())


,ubica_geo,nan_values
0,07012,891
1,14051,424
2,15012,1
3,15022,3
4,15073,489
5,15125,187
6,21128,189
7,21175,260
8,30074,2
9,30081,140


ideas to input missing values:
1. remove rows with missing values
2. fill with mean/median
3. fill with the most common value in the state
4. knn imputer using centroids of the municipalities

## S5: NaN Imputation

### Concentrado Hogar

In [72]:
# to new_index add nan_ubica_geo ubica_geo's
index_cols = ["ubica_geo", "year"]
new_index = pd.MultiIndex.from_product(
    [gdf_properties["ubica_geo"].unique(), gdf_properties["year_enigh"].unique()],
    names=index_cols
)

# reindex
df_concentradohogar_all = (
    df_concentradohogar_all
    .set_index(index_cols)
    .reindex(new_index)
    .reset_index()
    )

In [73]:
# see 31100
df_concentradohogar_all.query("ubica_geo == '31100'")

,ubica_geo,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares
1011,31100,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012,31100,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013,31100,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
# get cve_ent from ubica_geo
df_concentradohogar_all["cve_ent"] = df_concentradohogar_all["ubica_geo"].str[:2]

In [79]:
# get table of median values by cve_ent
table_concentradohogar_medians = (
    df_concentradohogar_all
    .drop(columns=["ubica_geo"])
    .groupby(["cve_ent", "year"], as_index=False)
    .mean()
)
table_concentradohogar_medians

,cve_ent,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares
0,01,2018,7.20,2.0,1.0,2.586135,2.331342,48.783703,5.743871,4.257246,2.121937,2.135309,3.254090,1.003157,2.980434,0.273656,1.955393,2.664982,1.915714,57003.504263,41681.567996,35902.188331,29509.240219,575.049637,470.237026,2178.863150,132.325245,1169.721151,1866.751904,4448.840622,3298.103376,601.307112,1302.303967,1394.492297,1150.737247,35.635823,1102.249488,12.851936,0.000000,1330.539042,2816.245228,2188.106441,628.138787,7807.205212,2908.670513,196.637780,989.725055,1089.355676,484.013078,1465.024028,673.779083,4682.537762,15.948064,35308.415772,11490.114078,8956.255259,1721.458645,1659.782878,93.022588,911.190521,302.462996,100.764355,145.429789,961.271191,348.633584,67.029367,41.581514,76.493094,1646.774376,880.360359,2452.662451,81.196368,1917.673989,1131.084290,786.589699,2456.622166,667.749256,121.500645,504.434768,1162.937496,2217.131515,1492.606818,223.306113,501.218584,1040.114815,771.435826,96.896386,171.782603,7875.721124,986.020634,205.237185,2373.235377,2984.130192,216.979411,2767.150781,1327.097736,4460.088873,2791.198760,949.075946,719.814167,2929.358896,2154.403129,129.779439,645.176328,921.590317,6502.857419,1816.592079,885.618371,2307.716148,9.054218,1483.876603,7707.718195,916.327180,790.878558,658.848444,132.030114,2062.212054,142.976718,898.248205,904.162830,424.560458,1568.352193,60702.20
1,01,2020,8.40,2.0,1.0,2.552323,2.338150,49.024817,5.993201,3.942444,1.911562,2.030882,3.122261,0.820183,2.878134,0.244128,1.879896,2.520096,1.816300,55607.465192,39529.635623,34917.052727,28762.862252,497.320507,638.114735,1912.699579,614.789694,939.792060,1551.473900,3269.108659,2694.511966,387.668448,1119.926874,1186.916644,574.596693,5.247301,569.337478,0.011915,0.000000,1343.474236,2140.055736,1832.356876,307.698860,9060.285523,3425.700574,77.654076,856.939056,1507.916854,863.449957,1628.962203,699.662803,4824.084565,53.403745,32658.931937,11966.251378,10399.257381,1846.769751,1897.050996,124.432911,972.562330,366.999857,107.685977,158.466069,1212.596605,500.884351,83.508337,59.233375,87.181438,1972.604194,1009.281189,1482.483549,84.510447,1271.090991,773.997413,497.093578,2637.636644,794.343337,153.137369,553.095445,1137.060492,2123.512545,1510.490290,197.044821,415.977434,1473.007048,1063.957994,151.258384,257.790671,6658.716677,612.850366,74.691228,1511.348658,2565.142486,267.545540,2297.596946,1894.683940,2955.457738,2043.645572,619.367061,292.445105,2648.634318,2098.746676,84.852098,465.035544,924.624598,5233.021834,2120.481024,453.993038,1049.556618,14.818160,1594.172994,9077.830528,851.922920,667.920633,267.581332,400.339301,4873.159292,131.430039,784.317997,690.069029,601.207445,477.803173,71725.00
2,01,2022,7.60,2.0,1.0,2.631701,2.327185,48.063382,6.178175,3.838164,1.866993,1.971171,2.964643,0.873521,2.744548,0.220094,1.739571,2.467944,1.726860,70876.362012,49432.641472,42941.309720,35180.076

In [105]:
# append to df_concentradohogar_all for nan values
mask = (
    df_concentradohogar_all['clase_hog'].isnull()
)
columns_to_impute = df_concentradohogar_all.drop(columns=["ubica_geo", "year", "cve_ent"]).columns.copy()

# append to df_concentradohogar_all for nan values
df_concentradohogar_all.loc[mask, columns_to_impute] = (
    df_concentradohogar_all
    .loc[mask, :]
    .merge(
        table_concentradohogar_medians,
        on=["cve_ent", "year"],
        how="left"
        )
    .filter(like="_y")
    .values
)

# sort by ubica_geo and year
df_concentradohogar_all = (
    df_concentradohogar_all
    .sort_values(["ubica_geo", "year"], ignore_index=True)
    )

# see how many nan values
df_concentradohogar_all.isnull().sum()[df_concentradohogar_all.isnull().sum() > 0]

Series([], dtype: int64)

In [106]:
# see 31100
df_concentradohogar_all.query("ubica_geo == '31100'")

,ubica_geo,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares,cve_ent
1002,31100,2018,524.0,2.0,1.0,2.425461,2.049813,47.532147,5.445398,3.839052,1.949956,1.889096,3.063356,0.775696,2.806116,0.257240,2.074376,2.622248,2.054416,47166.909216,33223.618704,28166.396221,23392.798150,255.884311,1383.336781,1217.078569,98.929386,417.411389,1400.957635,3619.342411,3541.411794,1127.999913,617.147370,1796.264511,77.930617,20.246164,25.267042,32.417411,0.0,1437.880071,3242.639994,2549.669459,692.970535,6668.996102,3177.021043,104.121556,1061.758079,18.372205,550.596280,1262.778869,494.348070,4008.765455,22.888960,30384.066068,11966.938045,10115.217337,1758.916783,2124.604219,74.465749,562.026725,321.081756,82.670991,95.775210,754.264073,223.667100,62.299581,87.795633,129.175719,2461.038767,1377.435030,1817.934786,33.785922,1068.692916,695.211072,373.481844,1856.746948,481.563559,54.309289,105.661346,1215.212754,1637.332664,1246.258383,98.390473,292.683808,1044.632368,784.320884,151.438181,108.873303,6570.661913,2099.763625,143.172816,938.252991,1900.918374,173.944348,1726.974025,1488.554108,3335.165337,2346.795740,820.341927,168.027671,1832.580439,1515.684307,99.381235,217.514898,1071.315437,3583.556207,1046.028682,630.052661,556.207632,24.715178,1326.552054,7332.809878,930.680680,208.738310,137.143710,71.594600,3246.158424,308.053374,918.910819,730.714674,178.170154,811.383443,72186.4,31
1003,31100,2020,538.2,2.0,1.0,2.366885,2.009724,48.066265,5.787704,3.652757,1.717995,1.934761,2.905382,0.747375,2.596984,0.308398,1.869483,2.482407,1.801681,40421.281691,27073.997257,22677.279845,18746.035760,240.811887,934.557073,1100.439085,189.314094,435.802725,1030.319221,2841.770013,2716.014952,532.017045,776.691897,1407.306010,125.755060,45.739422,49.120453,30.895185,0.0,1554.947400,1060.804380,659.193126,401.611254,7586.348131,3071.862462,59.693005,869.390254,22.717685,1161.888456,1201.896411,1198.899858,4682.265630,17.866292,26623.707247,11969.166748,10995.247926,1955.234344,2057.072077,70.886264,611.225621,372.877715,68.513312,109.320337,722.070243,313.985029,58.899916,103.206421,122.059310,3111.246486,1318.650851,952.610015,21.308807,397.633798,274.224499,123.409300,2006.511640,583.547567,104.844940,109.385843,1208.733290,1785.897601,1383.812674,82.993564,319.091363,1038.869924,713.176820,110.919327,214.773777,5096.703074,858.442957,37.081385,665.810543,1956.666767,224.987169,1731.679598,1578.701422,1668.108397,1073.126983,492.400075,102.581339,1805.032497,1434.010774,71.279100,299.742623,855.783568,4251.168373,1448.033354,688.736178,639.845547,13.979376,1460.573918,7786.162019,748.048888,492.414048,213.012353,279.401695,2960.324571,77.938185,1132.691689,1078.244246,781.183222,515.317170,73190.2,31
1004,31100,2022,542.0,2.0,1.0,2.428342,2.061257,48.833977,5.898544,3.519161,1.803041,1.716120,2.915355,0.603806,2.588247,0.327108,1.798277,2.394296,1.788719,57073.211817,39081.343509,33736.874887,

In [104]:
# see 31 in table
table_concentradohogar_medians.query("cve_ent == '31'")

,cve_ent,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares
90,31,2018,524.0,2.0,1.0,2.425461,2.049813,47.532147,5.445398,3.839052,1.949956,1.889096,3.063356,0.775696,2.806116,0.257240,2.074376,2.622248,2.054416,47166.909216,33223.618704,28166.396221,23392.798150,255.884311,1383.336781,1217.078569,98.929386,417.411389,1400.957635,3619.342411,3541.411794,1127.999913,617.147370,1796.264511,77.930617,20.246164,25.267042,32.417411,0.0,1437.880071,3242.639994,2549.669459,692.970535,6668.996102,3177.021043,104.121556,1061.758079,18.372205,550.596280,1262.778869,494.348070,4008.765455,22.888960,30384.066068,11966.938045,10115.217337,1758.916783,2124.604219,74.465749,562.026725,321.081756,82.670991,95.775210,754.264073,223.667100,62.299581,87.795633,129.175719,2461.038767,1377.435030,1817.934786,33.785922,1068.692916,695.211072,373.481844,1856.746948,481.563559,54.309289,105.661346,1215.212754,1637.332664,1246.258383,98.390473,292.683808,1044.632368,784.320884,151.438181,108.873303,6570.661913,2099.763625,143.172816,938.252991,1900.918374,173.944348,1726.974025,1488.554108,3335.165337,2346.795740,820.341927,168.027671,1832.580439,1515.684307,99.381235,217.514898,1071.315437,3583.556207,1046.028682,630.052661,556.207632,24.715178,1326.552054,7332.809878,930.680680,208.738310,137.143710,71.594600,3246.158424,308.053374,918.910819,730.714674,178.170154,811.383443,72186.4
91,31,2020,538.2,2.0,1.0,2.366885,2.009724,48.066265,5.787704,3.652757,1.717995,1.934761,2.905382,0.747375,2.596984,0.308398,1.869483,2.482407,1.801681,40421.281691,27073.997257,22677.279845,18746.035760,240.811887,934.557073,1100.439085,189.314094,435.802725,1030.319221,2841.770013,2716.014952,532.017045,776.691897,1407.306010,125.755060,45.739422,49.120453,30.895185,0.0,1554.947400,1060.804380,659.193126,401.611254,7586.348131,3071.862462,59.693005,869.390254,22.717685,1161.888456,1201.896411,1198.899858,4682.265630,17.866292,26623.707247,11969.166748,10995.247926,1955.234344,2057.072077,70.886264,611.225621,372.877715,68.513312,109.320337,722.070243,313.985029,58.899916,103.206421,122.059310,3111.246486,1318.650851,952.610015,21.308807,397.633798,274.224499,123.409300,2006.511640,583.547567,104.844940,109.385843,1208.733290,1785.897601,1383.812674,82.993564,319.091363,1038.869924,713.176820,110.919327,214.773777,5096.703074,858.442957,37.081385,665.810543,1956.666767,224.987169,1731.679598,1578.701422,1668.108397,1073.126983,492.400075,102.581339,1805.032497,1434.010774,71.279100,299.742623,855.783568,4251.168373,1448.033354,688.736178,639.845547,13.979376,1460.573918,7786.162019,748.048888,492.414048,213.012353,279.401695,2960.324571,77.938185,1132.691689,1078.244246,781.183222,515.317170,73190.2
92,31,2022,542.0,2.0,1.0,2.428342,2.061257,48.833977,5.898544,3.519161,1.803041,1.716120,2.915355,0.603806,2.588247,0.327108,1.798277,2.394296,1.788719,57073.211817,39081.343509,33736.874887,29681.421936,303.480831,650.720

### Vivendas

In [221]:
# to new_index add nan_ubica_geo ubica_geo's
index_cols = ["ubica_geo", "year"]
new_index = pd.MultiIndex.from_product(
    [gdf_properties["ubica_geo"].unique(), gdf_properties["year_enigh"].unique()],
    names=index_cols
)

# reindex
df_viviendas_all = (
    df_viviendas_all
    .set_index(index_cols)
    .reindex(new_index)
    .reset_index()
    )

In [222]:
# subset columns that are not duplicated
df_viviendas_all = (
    df_viviendas_all
    .loc[:, ~df_viviendas_all.columns.duplicated()]
)

In [223]:
# see 31100
df_viviendas_all.query("ubica_geo == '31100'")

,ubica_geo,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas
1011,31100,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012,31100,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013,31100,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
# get cve_ent from ubica_geo
df_viviendas_all["cve_ent"] = df_viviendas_all["ubica_geo"].str[:2]

In [225]:
# convert to float
cols_to_change_type = ["combustible", "tipo_finan", "tipo_viv", "mat_pisos"]
df_viviendas_all[cols_to_change_type] = df_viviendas_all[cols_to_change_type].astype(float)

In [226]:
# get table of median values by cve_ent
table_vivienda_medians = (
    df_viviendas_all
    .drop(columns=["ubica_geo"])
    .groupby(["cve_ent", "year"], as_index=False)
    .mean()
)
table_vivienda_medians

,cve_ent,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas
0,01,2018,8.0,10.0,2.80,1.0,3.0,3.0,4.0,2.20,4.2,1.0,1.0,1.0,0.912137,0.271362,0.800511,0.909509,0.0,4.420127,20.250340,1.112884,0.399239,0.967455,0.003587,2.187402,0.039379,230.706894,0.766226,1.670811,1.083465,0.042528,0.964255,2.331342,2.232725,1.146075,0.021342,0.005698,2.586135,0.082991,5.998591,0.301207,2.436640,1.079774,0.993295,0.102691,4.231534,0.407684,0.004955,0.243605,320.586506,0.264143,0.089692,0.013002,1.046560,1595.553006,60702.20
1,01,2020,8.0,10.0,3.00,1.0,3.0,3.0,4.0,1.80,4.2,1.0,1.0,1.0,0.943074,0.263799,0.795834,0.907967,0.0,3.981893,17.911069,1.152286,0.437911,0.984080,0.005726,1.926812,0.043178,268.739821,0.793554,1.201682,1.087265,0.031128,0.966095,2.338150,2.055080,1.132671,0.014753,0.007867,2.552323,0.113611,6.247882,0.318703,2.321090,1.063589,0.991903,0.107332,4.278451,0.365571,0.007148,0.193672,316.025198,0.294593,0.099538,0.008199,1.015828,1645.189596,71725.00
2,01,2022,8.0,10.0,3.00,1.0,3.0,2.6,3.8,1.80,3.4,1.0,1.0,1.0,0.955338,0.364700,0.843292,0.890058,0.0,3.914585,17.248517,1.176136,0.515727,0.970398,0.015353,1.903469,0.022253,339.712427,0.850006,0.795252,1.086765,0.036597,0.967262,2.327185,2.011116,1.232014,0.008596,0.006710,2.631701,0.107142,7.207891,0.313931,2.331273,1.083152,0.989237,0.104247,4.185339,0.413389,0.009488,0.145486,588.326450,0.361911,0.102420,0.021380,1.018819,2080.635767,72441.00
3,02,2018,8.0,8.5,2.25,1.0,3.0,1.0,4.0,1.50,5.0,1.0,1.0,1.0,0.852444,0.313906,0.101366,0.603249,0.0,3.370514,14.778483,0.976271,0.013631,0.974302,0.271139,1.682316,0.068284,495.536871,0.822640,1.190902,1.249026,0.059280,0.934396,2.450202,1.688198,0.994116,0.034741,0.002962,1.786824,0.050373,5.619687,0.042568,2.115245,1.394415,0.995097,0.127318,3.651680,0.542618,0.030859,0.331060,355.509805,0.063110,0.130529,0.006312,1.003140,2137.880791,266591.00
4,02,2020,8.0,8.5,2.50,1.0,3.0,1.0,4.0,2.00,5.0,1.0,1.0,1.0,0.913127,0.279764,0.108240,0.577239,0.0,3.309996,18.097982,1.099012,0.025186,0.979650,0.272621,1.613549,0.072284,519.522103,0.860794,0.819700,1.161200,0.088035,0.964287,2.485101,1.696447,1.115911,0.031589,0.001947,1.784814,0.062003,6.346981,0.043219,2.161368,1.252475,0.994902,0.170105,3.842881,0.588295,0.044019,0.279955,386.555622,0.059437,0.128789,0.021931,1.005444,2910.766735,272823.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,31,2020,8.0,10.0,3.00,1.0,2.6,1.0,4.0,1.40,3.4,1.0,1.0,1.0,0.861660,0.289199,0.820661,0.594545,0.0,3.731933,16.879545,1.001989,0.002678,0.905991,0.250149,1.754900,0.093957,204.002737,0.744071,0.348565,1.190140,0.049395,0.953110,2.009724,1.977033,0.991318,0.005589,0.012016,2.366885,0.022467,7.058724,0.092987,1.850504,2.133554,0.960583,0.073838,3.508086,0.105279,0.001947,0.136500,359.305775,0.229901,0.090638,0.074315,1.026010,1593.574772,73190.20
92,31,2022,8.0,10.0,2.80,1.0,2.6,1.0,4.0,1.40,2.6,1.0,1.0,1.0,0.862909,0.265821,0.831832,0.628512,0.0,3.580125,19.657799,0.988764,0.006678,0.908434,0.316408,1.840188,0.113051,180.334891,0.715371,0.178975,1.136088,0.022505,0.951691,2.061257,1.739937,1.008961,0.003557,0.008526,2.428342,0.038166,7.284024,0.093629,1.815024,2.019622,0.968428,0.113511,3.357695,0.097785,0.003806,0.144380,477.986526,0.134927,0.143175,0.067798,1.019311,2040.262618,75934.20
93,32,2018,8.0,10.0,2.75,1.0,3.0,1.0,4

In [227]:
mask.sum()

36

In [232]:
# append to df_viviendas_all for nan values
mask = (
    df_viviendas_all['mat_pared'].isnull()
)
columns_to_impute = df_viviendas_all.drop(columns=["ubica_geo", "year", "cve_ent"]).columns.copy()

# append to df_viviendas_all for nan values
df_viviendas_all.loc[mask, columns_to_impute] = (
    df_viviendas_all
    .loc[mask, :]
    .merge(
        table_vivienda_medians,
        on=["cve_ent", "year"],
        how="left"
        )
    .filter(like="_y")
    .values
)

# sort by ubica_geo and year
df_viviendas_all = (
    df_viviendas_all
    .sort_values(["ubica_geo", "year"], ignore_index=True)
    )

# see how many nan values
df_viviendas_all.isnull().sum()[df_viviendas_all.isnull().sum() > 0]

Series([], dtype: int64)

In [233]:
# see 31100
df_viviendas_all.query("ubica_geo == '31100'")

,ubica_geo,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas,cve_ent
1002,31100,2018,8.0,10.0,2.8,1.0,2.6,1.0,3.8,1.2,3.4,1.0,1.0,1.0,0.885791,0.272248,0.841223,0.502584,0.0,3.985898,18.885491,0.996769,0.002285,0.825177,0.213154,2.029412,0.090900,162.986406,0.703972,0.605953,1.137121,0.026766,0.980615,2.049813,1.956486,0.983030,0.001960,0.021820,2.425461,0.038560,6.567620,0.093586,1.909671,2.056778,0.969455,0.046804,3.437131,0.136885,0.000373,0.131111,340.937094,0.148716,0.131848,0.103535,1.045526,1366.720974,72186.4,31
1003,31100,2020,8.0,10.0,3.0,1.0,2.6,1.0,4.0,1.4,3.4,1.0,1.0,1.0,0.861660,0.289199,0.820661,0.594545,0.0,3.731933,16.879545,1.001989,0.002678,0.905991,0.250149,1.754900,0.093957,204.002737,0.744071,0.348565,1.190140,0.049395,0.953110,2.009724,1.977033,0.991318,0.005589,0.012016,2.366885,0.022467,7.058724,0.092987,1.850504,2.133554,0.960583,0.073838,3.508086,0.105279,0.001947,0.136500,359.305775,0.229901,0.090638,0.074315,1.026010,1593.574772,73190.2,31
1004,31100,2022,8.0,10.0,2.8,1.0,2.6,1.0,4.0,1.4,2.6,1.0,1.0,1.0,0.862909,0.265821,0.831832,0.628512,0.0,3.580125,19.657799,0.988764,0.006678,0.908434,0.316408,1.840188,0.113051,180.334891,0.715371,0.178975,1.136088,0.022505,0.951691,2.061257,1.739937,1.008961,0.003557,0.008526,2.428342,0.038166,7.284024,0.093629,1.815024,2.019622,0.968428,0.113511,3.357695,0.097785,0.003806,0.144380,477.986526,0.134927,0.143175,0.067798,1.019311,2040.262618,75934.2,31


In [234]:
# see 31 in table
table_vivienda_medians.query("cve_ent == '31'")

,cve_ent,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas
90,31,2018,8.0,10.0,2.8,1.0,2.6,1.0,3.8,1.2,3.4,1.0,1.0,1.0,0.885791,0.272248,0.841223,0.502584,0.0,3.985898,18.885491,0.996769,0.002285,0.825177,0.213154,2.029412,0.090900,162.986406,0.703972,0.605953,1.137121,0.026766,0.980615,2.049813,1.956486,0.983030,0.001960,0.021820,2.425461,0.038560,6.567620,0.093586,1.909671,2.056778,0.969455,0.046804,3.437131,0.136885,0.000373,0.131111,340.937094,0.148716,0.131848,0.103535,1.045526,1366.720974,72186.4
91,31,2020,8.0,10.0,3.0,1.0,2.6,1.0,4.0,1.4,3.4,1.0,1.0,1.0,0.861660,0.289199,0.820661,0.594545,0.0,3.731933,16.879545,1.001989,0.002678,0.905991,0.250149,1.754900,0.093957,204.002737,0.744071,0.348565,1.190140,0.049395,0.953110,2.009724,1.977033,0.991318,0.005589,0.012016,2.366885,0.022467,7.058724,0.092987,1.850504,2.133554,0.960583,0.073838,3.508086,0.105279,0.001947,0.136500,359.305775,0.229901,0.090638,0.074315,1.026010,1593.574772,73190.2
92,31,2022,8.0,10.0,2.8,1.0,2.6,1.0,4.0,1.4,2.6,1.0,1.0,1.0,0.862909,0.265821,0.831832,0.628512,0.0,3.580125,19.657799,0.988764,0.006678,0.908434,0.316408,1.840188,0.113051,180.334891,0.715371,0.178975,1.136088,0.022505,0.951691,2.061257,1.739937,1.008961,0.003557,0.008526,2.428342,0.038166,7.284024,0.093629,1.815024,2.019622,0.968428,0.113511,3.357695,0.097785,0.003806,0.144380,477.986526,0.134927,0.143175,0.067798,1.019311,2040.262618,75934.2


## S6: Re-Join with gdf_properties

### Load & Transform

In [235]:
# read parquet
gdf_properties = pd.read_parquet("../../data/interim/cleaned_data_s4.parquet")

# get original columns
original_columns = gdf_properties.columns

# see num of rows
gdf_properties.shape

(852931, 41)

In [236]:
# see columns cve ent & cve mun
gdf_properties[["id_entidad_f", "id_municipio"]]

,id_entidad_f,id_municipio
0,09,003
1,15,058
2,15,121
3,09,003
4,15,058
...,...,...
853028,31,041
853029,31,050
853030,31,050
853031,31,050


In [237]:
# create 'ubica_geo' column
gdf_properties["ubica_geo"] = (
    gdf_properties["id_entidad_f"].astype(str).str.zfill(2)
    + gdf_properties["id_municipio"].astype(str).str.zfill(3)
    )

# see some values
gdf_properties["ubica_geo"]

0         09003
1         15058
2         15121
3         09003
4         15058
          ...  
853028    31041
853029    31050
853030    31050
853031    31050
853032    31050
Name: ubica_geo, Length: 852931, dtype: object

In [238]:
# get year of fecha avaluo and floor every 2 years
gdf_properties["year_enigh"] = (
    gdf_properties["fecha_avaluo"].dt.year
    // 2 * 2
    )

# describe
gdf_properties["year_enigh"].describe()

count    852931.000000
mean       2020.196398
std           1.485187
min        2018.000000
25%        2020.000000
50%        2020.000000
75%        2022.000000
max        2022.000000
Name: year_enigh, dtype: float64

### Join with Concentrado Hogar

In [239]:
# join with df_concentradohogar_all
gdf_properties = (
    gdf_properties
    .merge(
        df_concentradohogar_all,
        left_on=["ubica_geo", "year_enigh"],
        right_on=["ubica_geo", "year"],
        how="left"
        )
    )

# see columns
gdf_properties.shape

(852931, 166)

In [240]:
# see size
gdf_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852931 entries, 0 to 852930
Columns: 166 entries, property_id to cve_ent
dtypes: datetime64[ns](1), float64(129), int32(2), int64(24), object(10)
memory usage: 1.0+ GB


In [241]:
# see if there are columns with "_y"
gdf_properties.filter(like="_x").columns

Index([], dtype='object')

In [244]:
# see how many nan values 
gdf_properties.isnull().sum()[gdf_properties.isnull().sum() > 0]

elevador           28371
cve_vigilancia    316447
tipo_vialidad     316447
dtype: int64

### Join with Vivienda

In [245]:
# join with df_viviendas_all
gdf_properties = (
    gdf_properties
    .drop(columns=["year", "tam_loc", "est_socio"])
    .merge(
        df_viviendas_all,
        left_on=["ubica_geo", "year_enigh"],
        right_on=["ubica_geo", "year"],
        how="left"
        )
    )

# see columns
gdf_properties.shape

(852931, 220)

In [246]:
# see size
gdf_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852931 entries, 0 to 852930
Columns: 220 entries, property_id to cve_ent_y
dtypes: datetime64[ns](1), float64(182), int32(2), int64(24), object(11)
memory usage: 1.4+ GB


In [247]:
# see if there are columns with "_y"
gdf_properties.filter(like="_x").columns

Index(['cve_ent_x'], dtype='object')

In [249]:
# drop cve_ent_x and cve_ent_y
gdf_properties = (
    gdf_properties
    .drop(columns=["cve_ent_x", "cve_ent_y"])
    )

In [251]:
# see how many nan values 
gdf_properties.isnull().sum()[gdf_properties.isnull().sum() > 0]

elevador           28371
cve_vigilancia    316447
tipo_vialidad     316447
dtype: int64

---
# Save

In [252]:
# save
dir_save = "../../data/interim/"
df_viviendas_all.to_csv(dir_save + "viviendas_enigh.csv")
df_concentradohogar_all.to_csv(dir_save + "concentrado_hogar_enigh.csv")

In [253]:
# save parquet
gdf_properties.to_parquet("../../data/interim/cleaned_data_s5_adamuz.parquet")